In [ ]:
# Testing some feature extraction large scale 

In [ ]:
import cv2
import numpy as np
from math import radians, cos, sin
import matplotlib.pyplot as plt

In [ ]:
# Import ground images and satellite images

# Images selected for local corrections
image_1 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9475.JPEG"
image_2 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9464.JPEG"
image_3 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9467.JPEG"
image_4 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9473.JPEG"
image_5 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9476.JPEG"
# Load in satellite reference image
sat_ref = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/SAT.png"

In [ ]:
image = cv2.imread(image_3)        # read the image from disk
plt.imshow(image)
# cv2.imshow("Image 1", image)       # show it in a window named "Image 1"
# cv2.waitKey(0)                     # wait for a key press
# cv2.destroyAllWindows()            # close the window

In [ ]:
def preprocess(img, edge_blur=1.0, canny_low=50, canny_high=150):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), edge_blur)
    edges = cv2.Canny(blurred, canny_low, canny_high)
    # Option: combine edges and gray: here we return both
    return gray, edges

def extract_sift(gray_or_edge, use_edges=False):
    if use_edges:
        inp = gray_or_edge
    else:
        inp = gray_or_edge
    sift = cv2.SIFT_create()
    kps, desc = sift.detectAndCompute(inp, None)
    return kps, desc

def root_sift(desc):
    # desc: uint8->float, L1 normalize then sqrt
    desc = desc.astype(np.float32)
    desc /= (desc.sum(axis=1, keepdims=True) + 1e-8)
    return np.sqrt(desc)

# FLANN matcher for SIFT
FLANN_INDEX_KDTREE = 1
flann = cv2.FlannBasedMatcher(dict(algorithm=FLANN_INDEX_KDTREE, trees=5), {})

def match_desc(d1, d2, ratio=0.7):
    if d1 is None or d2 is None:
        return []
    matches = flann.knnMatch(d1, d2, k=2)
    good = []
    for m,n in matches:
        if m.distance < ratio * n.distance:
            good.append(m)
    return good

def rotate_image(img, angle_deg):
    h,w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w/2,h/2), angle_deg, 1.0)
    return cv2.warpAffine(img, M, (w,h), flags=cv2.INTER_LINEAR)

def homography_ransac(kp1, kp2, matches, reprojThresh=3.0):
    if len(matches) < 4:
        return None, None
    ptsA = np.float32([kp1[m.queryIdx].pt for m in matches])
    ptsB = np.float32([kp2[m.trainIdx].pt for m in matches])
    H, status = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)
    return H, status


def match_with_rotation_search(ground_img, sat_img, use_edges=True,
                               rot_range=45, rot_step=5, ratio=0.7):
    # preprocess
    g_gray, g_edges = preprocess(ground_img)
    s_gray, s_edges = preprocess(sat_img)
    g_input = g_edges if use_edges else g_gray
    s_input = s_edges if use_edges else s_gray

    # extract descriptors for satellite once
    kp_s, desc_s = extract_sift(s_input)
    if desc_s is not None:
        desc_s = root_sift(desc_s)

    best = {'inliers':0, 'H':None, 'angle':0, 'matches':None, 'kp_g':None, 'kp_s':kp_s}
    # rotation search: rotate ground
    angles = np.arange(-rot_range, rot_range+1, rot_step)
    for ang in angles:
        g_rot = rotate_image(g_input, ang)
        kp_g, desc_g = extract_sift(g_rot)
        if desc_g is None:
            continue
        desc_g = root_sift(desc_g)
        good = match_desc(desc_g, desc_s, ratio=ratio)
        if len(good) < 8:
            continue
        H, status = homography_ransac(kp_g, kp_s, good, reprojThresh=4.0)
        if H is None:
            continue
        inliers = int(status.sum())
        if inliers > best['inliers']:
            best.update({'inliers': inliers, 'H': H, 'angle': ang, 'matches': good, 'kp_g':kp_g})
    return best



# Example usage:
ground_img = cv2.imread(image_3)
sat_tile = cv2.imread(sat_ref)
# print(sat_tile)


# Example:
result = match_with_rotation_search(ground_img, sat_tile, use_edges=True)
print('Best angle', result['angle'], 'inliers', result['inliers'])
if result['H'] is not None:
    # warp ground into sat frame for visualization
    g_warped = cv2.warpPerspective(rotate_image(preprocess(ground_img)[1], result['angle']), result['H'], (sat_tile.shape[1], sat_tile.shape[0]))
    cv2.imwrite('warped.png', g_warped)


In [ ]:
vis = cv2.drawMatches(rotate_image(preprocess(ground_img)[1], result['angle']),
                      result['kp_g'],
                      preprocess(sat_tile)[1],
                      result['kp_s'],
                      result['matches'], None,
                      matchesMask=None, flags=2)
cv2.imwrite('matches.png', vis)

In [ ]:

# ---------- STEP 1: Load images ----------
ground_img = cv2.imread(image_3)
sat_img = cv2.imread(sat_ref)

# Resize for easier visualization (optional)
scale = 0.5
ground_img = cv2.resize(ground_img, None, fx=scale, fy=scale)
sat_img = cv2.resize(sat_img, None, fx=scale, fy=scale)

# ---------- STEP 2: Select ground plane points ----------
# Manual selection for the prototype
print("Select 4 or more points on the GROUND image (press ENTER when done)")
g_points = []

def click_event_ground(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        g_points.append([x, y])
        cv2.circle(ground_img, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow("Ground Image", ground_img)

cv2.imshow("Ground Image", ground_img)
cv2.setMouseCallback("Ground Image", click_event_ground)
cv2.waitKey(0)
cv2.destroyAllWindows()

g_points = np.array(g_points, dtype=np.float32)
print(f"Selected ground points: {g_points}")

# ---------- STEP 3: Define ground-plane target coords ----------
# Define where these points should appear in the bird's-eye frame.
# For example, a 1000x1000 pixel rectangle for the ground plane
width, height = 1000, 1000
b_points = np.array([
    [0, 0],
    [width, 0],
    [width, height],
    [0, height]
], dtype=np.float32)

# Compute homography (ground image -> bird’s-eye)
H_ground2bird, _ = cv2.findHomography(g_points, b_points)

# Warp the image
bird_eye = cv2.warpPerspective(ground_img, H_ground2bird, (width, height))

cv2.imshow("Bird’s-eye View", bird_eye)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ---------- STEP 4: Feature matching between bird-eye and satellite ----------

# Convert to grayscale
gray_bird = cv2.cvtColor(bird_eye, cv2.COLOR_BGR2GRAY)
gray_sat = cv2.cvtColor(sat_img, cv2.COLOR_BGR2GRAY)

# SIFT features
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(gray_bird, None)
kp2, des2 = sift.detectAndCompute(gray_sat, None)

# Match using FLANN + ratio test
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(des1, des2, k=2)
good = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good.append(m)

# Draw initial matches
draw_params = dict(matchColor=(0,255,0), singlePointColor=None, flags=2)
img_matches = cv2.drawMatches(bird_eye, kp1, sat_img, kp2, good, None, **draw_params)
cv2.imshow("Initial Matches", img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ---------- STEP 5: Geometric verification ----------
if len(good) > 4:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
    H_bs, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matches_mask = mask.ravel().tolist()
    inliers = np.sum(matches_mask)

    print(f"Found {inliers} inliers out of {len(good)} matches")

    img_inliers = cv2.drawMatches(bird_eye, kp1, sat_img, kp2, good, None,
                                  matchColor=(0,255,0),
                                  singlePointColor=None,
                                  matchesMask=matches_mask, flags=2)
    cv2.imshow("RANSAC Inliers", img_inliers)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Not enough matches to compute homography.")
